In [66]:
#import libaries
import pandas as pd
import numpy as np 
import plotly.express as px 
from sklearn.impute import KNNImputer ,SimpleImputer
from datasist.structdata import detect_outliers


In [67]:
phalstine_df=pd.read_csv('../data/raw/fatalities_isr_pse_conflict_2000_to_2023.csv')
df=phalstine_df.copy()
df

,name,date_of_event,age,citizenship,event_location,event_location_district,event_location_region,date_of_death,gender,took_part_in_the_hostilities,place_of_residence,place_of_residence_district,type_of_injury,ammunition,killed_by,notes
0,'Abd a-Rahman Suleiman Muhammad Abu Daghash,9/24/2023,32.0,Palestinian,Nur Shams R.C.,Tulkarm,West Bank,9/24/2023,M,NaN,Nur Shams R.C.,Tulkarm,gunfire,live ammunition,Israeli security forces,Fatally shot by Israeli forces while standing ...
1,Usayed Farhan Muhammad 'Ali Abu 'Ali,9/24/2023,21.0,Palestinian,Nur Shams R.C.,Tulkarm,West Bank,9/24/2023,M,NaN,Nur Shams R.C.,Tulkarm,gunfire,live ammunition,Israeli security forces,Fatally shot by Israeli forces while trying to...
2,'Abdallah 'Imad Sa'ed Abu Hassan,9/22/2023,16.0,Palestinian,Kfar Dan,Jenin,West Bank,9/22/2023,M,NaN,al-Yamun,Jenin,gunfire,live ammunition,Israeli security forces,Fatally shot by soldiers while firing at them ...
3,Durgham Muhammad Yihya al-Akhras,9/20/2023,19.0,Palestinian,'Aqbat Jaber R.C.,Jericho,West Bank,9/20/2023,M,NaN,'Aqbat Jaber R.C.,Jericho,gunfire,live ammunition,Israeli security forces,Shot in the head by Israeli forces while throw...
4,Raafat 'Omar Ahmad Khamaisah,9/19/2023,15.0,Palestinian,Jenin R.C.,Jenin,West Bank,9/19/2023,M,NaN,Jenin,Jenin,gunfire,live ammunition,Israeli security forces,Wounded by soldiers’ gunfire after running awa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11119,Binyamin Herling,10/19/2000,64.0,Israeli,Nablus,Nablus,West Bank,10/19/2000,M,Israelis,Kedumim,Tulkarm,gunfire,live ammunition,Palestinian civilians,Killed while hiking on Mt. Eival.
11120,Farid Musa 'Issa a-Nesasreh,10/17/2000,28.0,Palestinian,Beit Furik,Nablus,West Bank,10/17/2000,M,Unknown,Beit Furik,Nablus,gunfire,NaN,Israeli civilians,Killed by a settler from Itamar while harvesti...
11121,Hillel Lieberman,10/7/2000,36.0,Israeli,Nablus,Nablus,West Bank,10/7/2000,M,Israelis,Elon Moreh,Nablus,gunfire,live ammunition,Palestinian civilians,His body was found a day after he disappeared.
11122,Fahed Mustafa 'Odeh Baker,10/7/2000,21.0,Palestinian,Bidya,Salfit,West Bank,10/7/2000,M,No,Bidya,Salfit,gunfire,NaN,Israeli civilians,Killed by settlers who rioted in Biddya village.


In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11124 entries, 0 to 11123
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   name                          11124 non-null  object 
 1   date_of_event                 11124 non-null  object 
 2   age                           10995 non-null  float64
 3   citizenship                   11124 non-null  object 
 4   event_location                11124 non-null  object 
 5   event_location_district       11124 non-null  object 
 6   event_location_region         11124 non-null  object 
 7   date_of_death                 11124 non-null  object 
 8   gender                        11104 non-null  object 
 9   took_part_in_the_hostilities  9694 non-null   object 
 10  place_of_residence            11056 non-null  object 
 11  place_of_residence_district   11056 non-null  object 
 12  type_of_injury                10833 non-null  object 
 13  a

In [69]:
df.isnull().sum()

name                               0
date_of_event                      0
age                              129
citizenship                        0
event_location                     0
event_location_district            0
event_location_region              0
date_of_death                      0
gender                            20
took_part_in_the_hostilities    1430
place_of_residence                68
place_of_residence_district       68
type_of_injury                   291
ammunition                      5253
killed_by                          0
notes                            280
dtype: int64

In [70]:
df=df.drop(columns='ammunition',axis=1)

In [71]:
df['date_of_death']=pd.to_datetime(df['date_of_death'])
df['date_of_event']=pd.to_datetime(df['date_of_event'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11124 entries, 0 to 11123
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   name                          11124 non-null  object        
 1   date_of_event                 11124 non-null  datetime64[ns]
 2   age                           10995 non-null  float64       
 3   citizenship                   11124 non-null  object        
 4   event_location                11124 non-null  object        
 5   event_location_district       11124 non-null  object        
 6   event_location_region         11124 non-null  object        
 7   date_of_death                 11124 non-null  datetime64[ns]
 8   gender                        11104 non-null  object        
 9   took_part_in_the_hostilities  9694 non-null   object        
 10  place_of_residence            11056 non-null  object        
 11  place_of_residence_district 

In [72]:
df.isnull().sum()/len(df)*100

name                             0.000000
date_of_event                    0.000000
age                              1.159655
citizenship                      0.000000
event_location                   0.000000
event_location_district          0.000000
event_location_region            0.000000
date_of_death                    0.000000
gender                           0.179791
took_part_in_the_hostilities    12.855088
place_of_residence               0.611291
place_of_residence_district      0.611291
type_of_injury                   2.615965
killed_by                        0.000000
notes                            2.517080
dtype: float64

In [73]:
col=['took_part_in_the_hostilities','notes','type_of_injury']
for i in col :
    df[i]=df[i].fillna('Unknown')
df.isnull().sum()/len(df)*100

name                            0.000000
date_of_event                   0.000000
age                             1.159655
citizenship                     0.000000
event_location                  0.000000
event_location_district         0.000000
event_location_region           0.000000
date_of_death                   0.000000
gender                          0.179791
took_part_in_the_hostilities    0.000000
place_of_residence              0.611291
place_of_residence_district     0.611291
type_of_injury                  0.000000
killed_by                       0.000000
notes                           0.000000
dtype: float64

In [74]:
df.dropna(inplace=True)

In [75]:
df.isnull().sum()

name                            0
date_of_event                   0
age                             0
citizenship                     0
event_location                  0
event_location_district         0
event_location_region           0
date_of_death                   0
gender                          0
took_part_in_the_hostilities    0
place_of_residence              0
place_of_residence_district     0
type_of_injury                  0
killed_by                       0
notes                           0
dtype: int64

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10951 entries, 0 to 11123
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   name                          10951 non-null  object        
 1   date_of_event                 10951 non-null  datetime64[ns]
 2   age                           10951 non-null  float64       
 3   citizenship                   10951 non-null  object        
 4   event_location                10951 non-null  object        
 5   event_location_district       10951 non-null  object        
 6   event_location_region         10951 non-null  object        
 7   date_of_death                 10951 non-null  datetime64[ns]
 8   gender                        10951 non-null  object        
 9   took_part_in_the_hostilities  10951 non-null  object        
 10  place_of_residence            10951 non-null  object        
 11  place_of_residence_district   109

In [77]:
df.citizenship.unique()

array(['Palestinian', 'Israeli', 'Jordanian', 'American'], dtype=object)

In [78]:
df.citizenship.value_counts()

citizenship
Palestinian    9951
Israeli         997
Jordanian         2
American          1
Name: count, dtype: int64

In [79]:
df.event_location_district.unique()

array(['Tulkarm', 'Jenin', 'Jericho', 'Gaza', 'Hebron', 'Tubas',
       'Ramallah and al-Bira', 'East Jerusalem', 'Nablus', 'Israel',
       'al-Quds', 'Bethlehem', 'Khan Yunis', 'Deir al-Balah',
       'North Gaza', 'Rafah', 'Qalqiliya', 'Salfit', 'Gaza Strip',
       'Gush Katif'], dtype=object)

In [80]:
df.event_location_district.value_counts()

event_location_district
Gaza                    2412
North Gaza              1889
Khan Yunis              1383
Rafah                   1053
Deir al-Balah            844
Israel                   661
Nablus                   631
Jenin                    501
Hebron                   337
Ramallah and al-Bira     337
Tulkarm                  243
Bethlehem                184
East Jerusalem           127
al-Quds                   84
Gush Katif                67
Qalqiliya                 64
Tubas                     50
Jericho                   44
Salfit                    36
Gaza Strip                 4
Name: count, dtype: int64

In [81]:
df.event_location_region.unique()

array(['West Bank', 'Gaza Strip', 'Israel'], dtype=object)

In [82]:
df.event_location_region.value_counts()

event_location_region
Gaza Strip    7652
West Bank     2638
Israel         661
Name: count, dtype: int64

In [83]:
df

,name,date_of_event,age,citizenship,event_location,event_location_district,event_location_region,date_of_death,gender,took_part_in_the_hostilities,place_of_residence,place_of_residence_district,type_of_injury,killed_by,notes
0,'Abd a-Rahman Suleiman Muhammad Abu Daghash,2023-09-24,32.0,Palestinian,Nur Shams R.C.,Tulkarm,West Bank,2023-09-24,M,Unknown,Nur Shams R.C.,Tulkarm,gunfire,Israeli security forces,Fatally shot by Israeli forces while standing ...
1,Usayed Farhan Muhammad 'Ali Abu 'Ali,2023-09-24,21.0,Palestinian,Nur Shams R.C.,Tulkarm,West Bank,2023-09-24,M,Unknown,Nur Shams R.C.,Tulkarm,gunfire,Israeli security forces,Fatally shot by Israeli forces while trying to...
2,'Abdallah 'Imad Sa'ed Abu Hassan,2023-09-22,16.0,Palestinian,Kfar Dan,Jenin,West Bank,2023-09-22,M,Unknown,al-Yamun,Jenin,gunfire,Israeli security forces,Fatally shot by soldiers while firing at them ...
3,Durgham Muhammad Yihya al-Akhras,2023-09-20,19.0,Palestinian,'Aqbat Jaber R.C.,Jericho,West Bank,2023-09-20,M,Unknown,'Aqbat Jaber R.C.,Jericho,gunfire,Israeli security forces,Shot in the head by Israeli forces while throw...
4,Raafat 'Omar Ahmad Khamaisah,2023-09-19,15.0,Palestinian,Jenin R.C.,Jenin,West Bank,2023-09-19,M,Unknown,Jenin,Jenin,gunfire,Israeli security forces,Wounded by soldiers’ gunfire after running awa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11119,Binyamin Herling,2000-10-19,64.0,Israeli,Nablus,Nablus,West Bank,2000-10-19,M,Israelis,Kedumim,Tulkarm,gunfire,Palestinian civilians,Killed while hiking on Mt. Eival.
11120,Farid Musa 'Issa a-Nesasreh,2000-10-17,28.0,Palestinian,Beit Furik,Nablus,West Bank,2000-10-17,M,Unknown,Beit Furik,Nablus,gunfire,Israeli civilians,Killed by a settler from Itamar while harvesti...
11121,Hillel Lieberman,2000-10-07,36.0,Israeli,Nablus,Nablus,West Bank,2000-10-07,M,Israelis,Elon Moreh,Nablus,gunfire,Palestinian civilians,His body was found a day after he disappeared.
11122,Fahed Mustafa 'Odeh Baker,2000-10-07,21.0,Palestinian,Bidya,Salfit,West Bank,2000-10-07,M,No,Bidya,Salfit,gunfire,Israeli civilians,Killed by settlers who rioted in Biddya village.


In [84]:
df['name'].value_counts()>1

name
Muhammad Mahmoud Rizeq Abu Taha          True
Ibrahim Sha'ban 'Abdallah al-Zein        True
Ahmad Radwan                             True
Iyad Salah 'Abd al-Hafez Qafah           True
'Ali Ahmad 'Ali al-'Aidy                 True
                                        ...  
'Abd a-Rahman Ibrahim Khalil a-Sarhi    False
Isma'il Muhammad Subhi Baker            False
Muhammad Ramez 'Izzat Baker             False
Farid Mahmoud 'Ali Abu Daqah            False
Wichlav Zalsevsky                       False
Name: count, Length: 10934, dtype: bool

In [85]:
len(df)

10951

In [86]:
df['name'].nunique()

10934

In [87]:
df['year']=df['date_of_death'].dt.year
df['month']=df['date_of_death'].dt.month
df['month_name']=df['date_of_death'].dt.month_name

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10951 entries, 0 to 11123
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   name                          10951 non-null  object        
 1   date_of_event                 10951 non-null  datetime64[ns]
 2   age                           10951 non-null  float64       
 3   citizenship                   10951 non-null  object        
 4   event_location                10951 non-null  object        
 5   event_location_district       10951 non-null  object        
 6   event_location_region         10951 non-null  object        
 7   date_of_death                 10951 non-null  datetime64[ns]
 8   gender                        10951 non-null  object        
 9   took_part_in_the_hostilities  10951 non-null  object        
 10  place_of_residence            10951 non-null  object        
 11  place_of_residence_district   109

In [89]:
df['age'] = df['age'].astype(int)

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10951 entries, 0 to 11123
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   name                          10951 non-null  object        
 1   date_of_event                 10951 non-null  datetime64[ns]
 2   age                           10951 non-null  int32         
 3   citizenship                   10951 non-null  object        
 4   event_location                10951 non-null  object        
 5   event_location_district       10951 non-null  object        
 6   event_location_region         10951 non-null  object        
 7   date_of_death                 10951 non-null  datetime64[ns]
 8   gender                        10951 non-null  object        
 9   took_part_in_the_hostilities  10951 non-null  object        
 10  place_of_residence            10951 non-null  object        
 11  place_of_residence_district   109

In [91]:
df.name.drop_duplicates()

0        'Abd a-Rahman Suleiman Muhammad Abu Daghash
1               Usayed Farhan Muhammad 'Ali Abu 'Ali
2                   'Abdallah 'Imad Sa'ed Abu Hassan
3                   Durgham Muhammad Yihya al-Akhras
4                       Raafat 'Omar Ahmad Khamaisah
                            ...                     
11119                               Binyamin Herling
11120                    Farid Musa 'Issa a-Nesasreh
11121                               Hillel Lieberman
11122                      Fahed Mustafa 'Odeh Baker
11123                              Wichlav Zalsevsky
Name: name, Length: 10934, dtype: object

To Save In Processed Folder

In [92]:
df.to_csv('../data/processed/datset_cleaned.csv')

In [93]:
df.gender.value_counts()

gender
M    9565
F    1386
Name: count, dtype: int64

In [94]:
df.year.min() , df.year.max()

(2000, 2023)

In [95]:
df.year.value_counts()

year
2014    2293
2002    1247
2009    1053
2004     921
2008     895
2003     723
2006     673
2007     400
2001     334
2021     320
2018     300
2012     261
2023     248
2005     238
2022     205
2015     178
2019     144
2011     127
2016     116
2010      89
2017      80
2013      42
2000      33
2020      31
Name: count, dtype: int64

In [96]:
df.name.value_counts().describe()

count    10934.000000
mean         1.001555
std          0.039402
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          2.000000
Name: count, dtype: float64